In [ ]:
# One trial is one sentence
# Pupil dilation for every time the user looks at the next phrase, or at the end at the quit key (some people do not 
# finish a sentence)

# TODO: Add changes when user might think its almost correct -- MSD is low

In [ ]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt

pd.options.mode.use_inf_as_na = True


In [ ]:
KeyInclude = ['IncreaseTimeDwell', 'DecreaseTimeDwell']
TimeDwellOrig = 800
TimeFixation = 300

In [ ]:
def ComputeDwellTime(userKeys):
    #print(userKeys)
    TimeDwellChanges = [key for key in userKeys if key[1] in KeyInclude]
    #print(TimeDwellChanges)
    
    if TimeDwellChanges:
        for pctChange in TimeDwellChanges:
            if pctChange[2] == 1:
                if pctChange[1] == 'IncreaseDwellTime':
                    TimeDwellNew = TimeDwellNew + 100
                else:
                    TimeDwellNew = TimeDwellNew - 100
    else:
        TimeDwellNew = TimeDwellOrig
    #print(TimeDwellNew)   
    return TimeDwellNew

In [ ]:
# Create list of list (made of epochs) composed of times when user looked at specific key, and only keep the ones that were
# activated

def UserLookedAtKey(keysLookedAt, keySpecify):
    epoch = list()   
    epochList = list()
    
    keyTime = [key for key in keysLookedAt if keySpecify in key[1]]
    keySpecifyInd = [keyInd[0] for keyInd in enumerate(keysLookedAt) if keySpecify in keysLookedAt[keyInd[0]][1]]
    keyIndArray = np.diff(np.asarray(keySpecifyInd))
    
    # TODO: reduce the list to only the first and the last elements of the epoch    
    
    for keyInd in enumerate(keyIndArray):
        if keyInd[0] == 0:
            epoch.append(keyTime[0])
            continue
        
        if keyInd[1] < 5:
            epoch.append(keyTime[keyInd[0]+1])
        else:
            epochList.append(epoch)
            epoch = list()    
            epoch.append(keyTime[keyInd[0]+1])
    if epoch:
        epochList.append(epoch)
    
    #print(len(epochList))
    countOne = 0
    listItemInd = len(epochList)-1
    
    while listItemInd > -1:
        flagOne = 0
        flagTwo = 0
        for row in epochList[listItemInd]:
            countOne = countOne + row.count('1')
            if '1' in row and flagOne == 0:
                #print(row)
                #print('flagOne', flagOne)
                flagOne = 1
                rowSplit = epochList[listItemInd].index(row) + 1
                continue
                
            if '1' in row and flagOne == 1:
                #print(row)
                #print('flagOne', flagOne)
                #print('flagTwo', flagTwo)
                flagTwo = 1
                
                
        if flagOne != 1:
            epochList.remove(epochList[listItemInd])
        
        if flagTwo == 1:
            epochList.insert(listItemInd, epochList[listItemInd][0:rowSplit])
            epochList.insert(listItemInd+1, epochList[listItemInd][rowSplit:-1])
            epochList.remove(epochList[listItemInd+2])
            
        
        listItemInd = listItemInd - 1
    
    #print(epochList[1])
    #print('next')
    #print(epochList[2])
    #print(countOne)
    return epochList

In [ ]:
# function to convert list of date and time into datetime format list

def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [ ]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [ ]:
# function to remove all invalid data in gazelog
def cleanGazeLog(gazeLog):
    gazeLogNew = list()
    for rowInd in enumerate(gazeLog):
        if 'Invalid' not in gazeLog[rowInd[0]]:
            gazeLogNew.append(gazeLog[rowInd[0]])
    return gazeLogNew

In [ ]:
def hampel(vals_orig, k, sd):
    '''
    vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)
    '''
    # Obtained from: https://stackoverflow.com/questions/46819260/filtering-outliers-how-to-make-median-based-
    # hampel-function-faster
    
    #Make copy so original not edited
    vals = pd.DataFrame(vals_orig)  
    vals = vals.replace([np.inf, -np.inf], np.nan).interpolate('linear', limit_area = 'inside') # linear interpolation instead 
    # simply copying the previous value --\ linear interpolation than cubic to not add any patterns in the data
    
    #vals = vals.replace([np.inf, -np.inf], np.nan).fillna(vals.rolling(window=k, min_periods=1, center=True).median())
    #vals = vals.replace([np.inf, -np.inf], np.nan).fillna(method='ffill')
    L= 1.4826
    rolling_median = vals.rolling(window=k, min_periods=1, center=True).median()
    difference = np.abs(rolling_median-vals)
    median_abs_deviation = difference.rolling(k).median()
    threshold = sd * L * median_abs_deviation
    outlier_idx = difference>threshold
    vals[outlier_idx] = rolling_median[outlier_idx]
    #print(vals)
    #valsWoNan = vals.replace([np.inf, -np.inf], np.nan).dropna()
    return(vals)

In [ ]:
def phraseTyped(phraseUser, phraseStim, nextPhrase):
    phraseUserEnd = list()
    
    # create list of end phrases typed
    for row_ind in range(0, len(phraseUser)):
        if row_ind!= 0 and phraseUser[row_ind][1] == '':
            phraseUserEnd.append(phraseUser[row_ind-1])
               
    # Attempt to remove all elements that are not end phrases - stimulated or user-generated

    for row_ind2 in range(0, len(phraseStim)):
        if phraseStim[row_ind2][1] == 'THE EXPERIMENT IS NOW DONE':
            if len(phraseUserEnd) != row_ind2:
                phraseUserEnd.remove(phraseUserEnd[row_ind2])
            phraseStim.remove(phraseStim[row_ind2])
            nextPhrase.remove(nextPhrase[row_ind2])
    
    row_ind3 = len(phraseUserEnd)-1
    while row_ind3 > -1:
        if phraseUserEnd[row_ind3][1] == '':
            if row_ind3 < len(phraseStim):
                phraseStim.remove(phraseStim[row_ind3])
                phraseUserEnd.remove(phraseUserEnd[row_ind3])
                nextPhrase.remove(nextPhrase[row_ind3])
        row_ind3 = row_ind3 - 1
    
    # if phrases documented have a comma, the said phrase is split and saved into different columns. They need to be joined
    for row_ind4 in range(0, len(phraseStim)):
        if len(phraseStim[row_ind4])>2:
            phraseStim[row_ind4][1] = ''.join(phraseStim[row_ind4][1:])
        
    for row_ind5 in range(0, len(phraseUserEnd)):
        if len(phraseUserEnd[row_ind5])>2:
            phraseUserEnd[row_ind5][1] = ''.join(phraseUserEnd[row_ind5][1:])
    
    return phraseUserEnd, phraseStim, nextPhrase

In [ ]:
def findScratchPadText(trialEpoch, scratchPadList, phraseList, GazeLog, TimeDwell, subjName):
    
    flagFirst = 0
    rowTimeList = -1
    timeUserLooked = list()
   
    phraseScratchedInd = 0
    phraseToBeScratchedInd = 0
    Veracity = list()
    
    timeStrUserLooked =  [[itemTime[0] for itemTime in item] for item in trialEpoch]
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    
    timeUserLooked = [timeConversion(timeStr) for timeStr in timeStrUserLooked]
    timeGazeLog = timeConversion(timeStrGazeLog)
        
    # Create list of pupil sizes from gazelog
    pupilLogL = [float(item4[29]) if 'Invalid' not in item4 else np.nan for item4 in GazeLog]
    pupilLogR = [float(item5[31]) if 'Invalid' not in item5 else np.nan for item5 in GazeLog]

    fig = plt.figure()
    axL = fig.add_subplot(2, 1, 1)
    axR = fig.add_subplot(2, 1, 2)
    
    for timeList in timeUserLooked:
        rowTimeList = rowTimeList + 1
        # Extract epoch data from gazelog
        
        timeScratchPadActive = timeList[0] - datetime.timedelta(milliseconds=float(trialEpoch[rowTimeList][0][2])*TimeDwell) - datetime.timedelta(milliseconds=TimeFixation)
        timeGazeLogStart, GazeLogStartInd = nearestTimePoint(timeGazeLog, timeScratchPadActive)
        
        timeGazeLogEnd, GazeLogEndInd = nearestTimePoint(timeGazeLog, timeList[-1])
        #print(timeScratchPadActive, timeGazeLogStart, timeList[-1], timeGazeLogEnd)
        
        if GazeLogStartInd != GazeLogEndInd:
            gazeEpochL = pupilLogL[GazeLogStartInd:GazeLogEndInd]
            gazeEpochR = pupilLogR[GazeLogStartInd:GazeLogEndInd]
            
            # Filter pupil sizes
            pupilWoOutlierL = hampel(gazeEpochL, 9, 3)
            pupilWoOutlierR = hampel(gazeEpochR, 9, 3)
            
            #print(pupilWoOutlierR)
            
            # Moving Mean of data without outliers: 
            pupilMeanL = pupilWoOutlierL.rolling(window=9, min_periods=1, center=True).mean()
            pupilMeanR = pupilWoOutlierR.rolling(window=9, min_periods=1, center=True).mean()
            
            # After the filtering is done: CHANGE THIS
            pupilSizeL = pupilWoOutlierL.mean(numeric_only=float)[0]
            pupilSizeR = pupilWoOutlierR.mean(numeric_only=float)[0] 
            
            pupilPlotL = (pupilMeanL - pupilMeanL[0][0])/pupilMeanL[0][0]
            
            pupilPlotR = (pupilMeanR - pupilMeanR[0][0])/pupilMeanR[0][0]
            
            #pupilL = pupilL + pupilPlotL
            #pupilR = pupilR + pupilPlotR
            
        else:
            print('one')
            continue
            #continue # remove samples with only 1 sample
        
        #print(pupilPlotL)
        # Also to check if phrase typed is correct or not, only check the last element of epoch
        #print(timeList[-1])
   
        
        phraseScratched = scratchPadList[rowTimeList][1]
        
        phraseToBeScratched = phraseList[rowTimeList][1]
        
        # are they the same? yes/no -> save in list with the time
        if phraseScratched == phraseToBeScratched:
            Veracity.append([pupilSizeL, pupilSizeR, 'C'])
            x = np.arange(0, float(len(pupilPlotL)/90), float(1/90))
            axL.plot(rowTimeList, pupilSizeL, 'bo', label = rowTimeList)
            axR.plot(rowTimeList, pupilSizeR, 'bo', label = rowTimeList)
            #axL.plot(x, pupilPlotL, '--', label = rowTimeList)
            #axR.plot(x, pupilPlotR, '--', label = rowTimeList)
        else:
            Veracity.append([pupilSizeL, pupilSizeR, 'INC'])
            x = np.arange(0, float(len(pupilPlotL)/90), float(1/90))
            
            axL.plot(rowTimeList, pupilSizeL, 'ro', label = rowTimeList)
            axR.plot(rowTimeList, pupilSizeR, 'ro', label = rowTimeList)
            
            #axL.plot(x, pupilPlotL, ':', label = rowTimeList)
            #axR.plot(x, pupilPlotR, ':', label = rowTimeList)
            
    
    axL.set_title('%s : Left' %subjName)
    axL.set_xlabel('Trials')
    axL.set_ylabel('Relative Pupil dilation [a.u.]')
    axL.legend()
    axR.set_title('%s : Right' %subjName)
    axR.set_xlabel('Trials')
    axR.set_ylabel('Relative Pupil dilation [a.u.]')
    axR.legend()
    plt.tight_layout()
    
    return Veracity

In [ ]:
subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0
for root, dirs, subfolder in os.walk(subjName):
    LetterLookedAtList = list()
    LetterLookedAt = list()
    
    if not dirs:
        
        if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root or 'jg' in root:
            continue
            
        userKeys = None
        scratchPad = None
        gazeLog = None
        stimPhrase = None
        print(root)
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'user_look*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'ScratchPad*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)  
                    scratchPad.remove(scratchPad[0])
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'PhraseLog*'):
                try:
                    fStimPhrase = open(root + '\\' + file, encoding='utf-8')
                    readerStimPhrase = csv.reader(fStimPhrase)
                    stimPhrase = list(readerStimPhrase)
                    stimPhrase.remove(stimPhrase[0])
                except:
                    if fStimPhrase is not None:
                        fStimPhrase.close()
                    else:
                        print('error in opening the phrase log file')
            elif fnmatch.fnmatch(file, 'GazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
            
                # if all these lists exist
            if userKeys is None or scratchPad is None or stimPhrase is None or gazeLog is None:
                continue
            else:     
                # Compute dwell time
                TimeDwell = ComputeDwellTime(userKeys)
            
                # call function to check when scratchpad is looked at and save it in a list
                nextPhraseKeyTime = UserLookedAtKey(userKeys, 'NextPhrase')
                quitKeyTime = UserLookedAtKey(userKeys, 'Quit')
            
                quitKeyTime = [quitKeyTime[-1]]
                
                nextPhraseTimesMonitor = nextPhraseKeyTime+quitKeyTime
                # For quit, check if the key is selected
    
                # for every element, find the time closest and previous to it, and check what was typed 
                # AND what should have been typed
                # Also, add gaze data to epoch, but first subtract the fixation time and complete the pupil data filtering as
                # per Per's paper
                
                # Find phrases typed in the end:
                phraseScratchPadC, phraseStimC, nextPhraseTimesMonitorC= phraseTyped(scratchPad, stimPhrase, nextPhraseTimesMonitor)
                
                a = re.compile('(?<=TypingData)(.*)(?=OptiKey)')
                subjName = a.findall(root)[0]
                #print(subjName)
                WrittenTruthListAll = findScratchPadText(nextPhraseTimesMonitorC, phraseScratchPadC, phraseStimC, gazeLog, TimeDwell, subjName)
                #print('done')
                break
    